## Load bioactivity data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("bioactivity_preprocessed_data.csv")

## Calculate Lipinski descriptors

Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excertion (ADME) that is also known as the pharmacokinteic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

The Lipinski's Rule stated the following:
- molecular weight < 500 Dalton,
- octanol-water partition coefficent (LogP) < 5,
- hydrogen bond donors < 5,
- hydrogen bond acceptors < 10.

## Import libraries

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

## Calculate descriptors

In [ ]:
def lipinski(smiles, verbose=False):
    moldata = []
    
    for element in smiles:
        mol = Chem.MolFromSmiles(element)
        moldata.append(mol)
        
    base_data = np.arange(1, 1)
    i = 0
    
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
        
        row = np.array([desc_MolWt, desc_MolLogP, desc_NumHDonors, desc_NumHAcceptors])
        
        if(i == 0):
            base_data = row
        else:
            base_data = np.vstack([base_data, row])
        
        i = i + 1
        
    column_names = ["MW", "LogP", "NumHDonors", "NumHAcceptors"]
    descriptors = pd.DataFrame(data=base_data, columns=column_names)
    
    return descriptors

In [ ]:
df_lipinski = lipinski(df.canonical_smiles)

## Combine dataframes

Let's take a look at the 2 dataframes that will be combined.

In [ ]:
df_lipinski

In [ ]:
df

Now, let's combine these 2 dataframes.

In [ ]:
df_combined = pd.concat([df, df_lipinski], axis=1)

In [ ]:
df_combined

## Convert IC50 to pIC50

To allow IC50 data to be more uniformly distributed, we will convert IC50 to the negative logarithmic scale which is essentially -log10(IC50). This custom function pIC50() will accept a dataframe as input and will:
- take the IC50 values from the standard_values column and converts it from nM to M by multiplying the value by 10**-9,
- take the molar value and apply -log10,
- delete the standard_value column and create a new pIC50 column.

In [ ]:
import numpy as np

def pIC50(input):
    pIC50 = []
    
    for i in input["standard_value_norm"]:
        # Converts nM to M
        molar = i * (10**-9)
        pIC50.append(-np.log10(molar))
        
    input["pIC50"] = pIC50
    x = input.drop("standard_value_norm", 1)
    
    return x

Point to note: values greater than 100kk will be fixed at 100kk otherwise the nagative logarithmic value will become negative.

In [ ]:
df_combined.standard_value.describe()

In [ ]:
def norm_value(input):
    norm = []
    
    for i in input["standard_value"]:
        if i > 100000000:
            i = 100000000
        norm.append(i)
        
    input["standard_value_norm"] = norm
    x = input.drop("standard_value", 1)
    
    return x

We will first apply the norm_value() function so that the values in the standard_value column is normalized.

In [ ]:
df_norm = norm_value(df_combined)

In [ ]:
df_norm

In [ ]:
df_norm.standard_value_norm.describe()

In [ ]:
df_final = pIC50(df_norm)

In [ ]:
df_final

In [ ]:
df_final.pIC50.describe()

## Removing the intermediate bioactivity class

Here, we will be removing the intermediate class from our data set.

In [ ]:
df_2class = df_final[df_final.bioactivity_class != "intermediate"]

In [ ]:
df_2class

## Exploratory data analysis (Chemical Space Analysis) via Lipinski descriptors

In [ ]:
# Import libraries
import seaborn as sns
sns.set(style="ticks")
import matplotlib.pyplot as plt

### Frequency plot of the two bioactivity classes

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.countplot(x="bioactivity_class", data=df_2class, edgecolor="black")

plt.xlabel("Bioactivity class", fontsize=14, fontweight="bold")
plt.ylabel("Frequency", fontsize=14, fontweight="bold")

plt.savefig("plot_bioactivity_class.pdf")

## Scatter plot of MW versus LogP

It can be seen that the two bioactivity classes are spanning similar chemical spaces as evident by the scatter plot of MW vs LogP.

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.scatterplot(x="MW", y="LogP", data=df_2class, hue="bioactivity_class", size="pIC50", edgecolor="black", alpha=0.7)

plt.xlabel("MW", fontsize=14, fontweight="bold")
plt.ylabel("LogP", fontsize=14, fontweight="bold")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig("plot_MW_vs_LogP.pdf")

## Box plots

## pIC50 value

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x="bioactivity_class", y="pIC50", data=df_2class)

plt.xlabel("Bioactivity class", fontsize=14, fontweight="bold")
plt.ylabel("pIC50 value", fontsize=14, fontweight="bold")

plt.savefig("plot_ic50.pdf")

## Statistical analysis | Mann-Whitney U Test

In [ ]:
def mann_whitney(descriptor, verbose=False):
    from numpy.random import seed, randn
    from scipy.stats import mannwhitneyu
    
    # Seed the random number generator
    seed(1)
    
    # Actives and inactives
    selection = [descriptor, "bioactivity_class"]
    df = df_2class[selection]
    active = df[df.bioactivity_class == "active"]
    active = active[descriptor]
    
    selection = [descriptor, "bioactivity_class"]
    df = df_2class[selection]
    inactive = df[df.bioactivity_class == "inactive"]
    inactive = inactive[descriptor]
    
    # Compare samples
    stat, p = mannwhitneyu(active, inactive)
    
    # Interpret
    alpha = 0.05
    
    if p > alpha:
        interpretation = "Same distribution (fail to reject H0)"
    else:
        interpretation = "Different distribution (reject H0)"
        
    results = pd.DataFrame({"Descriptor": descriptor,
                            "Statistics": stat,
                            "p": p,
                            "alpha": alpha,
                            "Interpretation": interpretation}, index=[0])
    filename = "mann_whitney_u_" + descriptor + ".csv"
    results.to_csv(filename)
    
    return results

In [ ]:
mann_whitney("pIC50")

## MW

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x="bioactivity_class", y="MW", data=df_2class)

plt.xlabel("Bioactivity class", fontsize=14, fontweight="bold")
plt.ylabel("MW", fontsize=14, fontweight="bold")

plt.savefig("plot_MW.pdf")

In [ ]:
mann_whitney("MW")

## LogP

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x="bioactivity_class", y="LogP", data=df_2class)

plt.xlabel("Bioactivity class", fontsize=14, fontweight="bold")
plt.ylabel("LogP", fontsize=14, fontweight="bold")

plt.savefig("plot_LogP.pdf")

In [ ]:
mann_whitney("LogP")

## NumHDonors

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x="bioactivity_class", y="NumHDonors", data=df_2class)

plt.xlabel("Bioactivity class", fontsize=14, fontweight="bold")
plt.ylabel("NumHDonors", fontsize=14, fontweight="bold")

plt.savefig("plot_NumHDonors.pdf")

In [ ]:
mann_whitney("NumHDonors")

## NumHAcceptors

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x="bioactivity_class", y="NumHAcceptors", data=df_2class)

plt.xlabel("Bioactivity class", fontsize=14, fontweight="bold")
plt.ylabel("NumHAcceptors", fontsize=14, fontweight="bold")

plt.savefig("plot_NumHAcceptors.pdf")

In [ ]:
mann_whitney("NumHAcceptors")

## Interpretation of Statistical Results

### Box Plots

### pIC50 values
Taking a look at pIC50 values, the actives and inactives displayed statistically significant difference, which is to be expected since threshold values (IC50 < 1k nM = Actives while IC50 > 10k nM = Inactives, corresponding to pIC50 > 6 = Acives and pIC50 <5 = Inactives) were used to define actives and inactives.

### Lipinski's descriptors
Of the four Lipinski's descriptors (MW, LogP, NumHDonors, NumHAcceptors), only LogP exhibited no difference between the actives and inactives while the other 3 descriptors shows statistically significant difference between actives and inactives.

## Zip result files

In [ ]:
! zip -r results.zip . -i *.csv *.pdf